In [1]:
pip install Pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=35bfebc8a368c1fa315f0804246412a373265c17f9c82377a33d87b69dad33b7
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built Pyspark


In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('CustomerChurn').getOrCreate()

In [5]:
df =spark.read.csv('customer_churn.csv',header =True,inferSchema =True)

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [8]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols =['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [12]:
output =assembler.transform(df)

In [13]:
final_data = output.select('features','churn')

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [14]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr = LogisticRegression(labelCol='churn')

# Fit the model
lrModel = lr.fit(train_data)

In [18]:
model_data=lrModel.summary

In [21]:
model_data.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                646|                646|
|   mean|0.16718266253869968|0.12538699690402477|
| stddev| 0.3734280177117469| 0.3314138218587594|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
pred_labels=lrModel.evaluate(test_data)
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.71166308484414...|[0.99109028238762...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.70602332296353...|[0.99877797516485...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.65266117188975...|[0.99650409571894...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.46016301873787...|[0.81155760681082...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.67768818985649...|[0.97534203373198...|       0.0|
|[31.0,5387.75,0.0...|    0|[2.83263843193427...|[0.94441427159341...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.48255153856771...|[0.98882183159188...|       0.0|
|[31.0,9574.89,0.0...|    0|[3.63472899696481...|[0.97428749542358...|       0.0|
|[32.0,8617.98,1.0...|    1|[0.99291181491224...|[0.72966267651226...|       0.0|
|[32.0,9885.12,1

In [28]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [29]:
auc =churn_eval.evaluate(pred_labels.predictions)

In [30]:
auc

0.7859389038634322

In [31]:
final_lr_model =lr.fit(final_data)

In [32]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header =True)

In [34]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [35]:
test_new_customers = assembler.transform(new_customers)

In [36]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
final_results = final_lr_model.transform(test_new_customers)

In [42]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

